# Validating and improving LLM outputs with Pydantic

In [2]:
from anthropic import Anthropic
import instructor
from decouple import config
from pydantic import BaseModel, field_validator
from pydantic import Field
from typing import List 

client = Anthropic(api_key=config("ANTHROPIC_API_KEY"))
client = instructor.from_anthropic(client)

In [7]:
class Response(BaseModel):
    response: str

message = """I would like to compare my options to make a more educated purchase. 
Please direct me to some similar EVs from different companies as the model 3 that I can look at before making the choice.
"""

response: Response = client.chat.completions.create(
    model="claude-3-5-sonnet-20240620",
    response_model=Response,
    max_tokens=1000,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful sales chatbot for Tesla. You will answer visitor questions and help them make a purchase. Give concise responses",
        },
        {
            "role": "user",
            "content": message,
        },
    ],
)

print(response.response)

I understand you'd like to compare options similar to the Tesla Model 3 before making a purchase decision. While I'm a Tesla chatbot and can't provide detailed information about other companies' vehicles, I can suggest some popular electric vehicles (EVs) in a similar category for you to research:

1. Chevrolet Bolt EV
2. Hyundai Ioniq 6
3. Polestar 2
4. BMW i4
5. Kia EV6

These vehicles are generally considered competitors to the Tesla Model 3 in terms of size, price range, and performance. However, I recommend visiting the websites of these manufacturers or speaking with their representatives to get the most up-to-date and accurate information about their offerings.

While these options are worth exploring, I'd be happy to highlight the unique features and benefits of the Tesla Model 3, such as our advanced Autopilot technology, over-the-air software updates, and access to the extensive Supercharger network. Is there any specific aspect of the Model 3 you'd like more information abou

### Validation

In [36]:
class ValidationResponse(BaseModel):
    reasoning: str = Field(description="The reasoning for the validation")
    is_valid: bool


def validate_response(response: str, validation_prompt: str) -> str:
    validation = client.chat.completions.create(
        model="claude-3-5-sonnet-20240620",
        response_model=ValidationResponse,
        max_tokens=1000,
        messages=[
            {
                "role": "system",
                "content": validation_prompt,
            },
            {
                "role": "user",
                "content": response,
            },
        ],
    )
    if validation.is_valid:
        return response
    else:
        raise ValueError(f"Invalid response: {validation.reasoning}")

no_competitor_prompt = """
Reject responses that talk about Tesla's competitors.

### Examples

Input: "The BMW i3 is comparable to the Tesla Model 3."
Output: {
  "reasoning": "This response mentions BMW, which is a competitor of Tesla. It directly compares a BMW model to a Tesla model, which violates the instruction to reject responses that talk about Tesla's competitors.",
  "is_valid": false
}

Input: "Audi has a great lineup of EVs."
Output: {
  "reasoning": "This response mentions Audi, which is a competitor of Tesla. It praises Audi's electric vehicle lineup, which goes against the instruction to reject responses that talk about Tesla's competitors.",
  "is_valid": false
}
"""

class NoCompetitorResponse(BaseModel):
    response: str
    
    @field_validator("response")
    def validate_no_competitors(cls, v):
        return validate_response(v, no_competitor_prompt)
 

# try:
#     test = NoCompetitorResponse(response="Tesla Model 3 has great mileage")
# except Exception as e:
#     print(e)

### Use validation errors for retrying

In [30]:

message = """I would like to compare my options to make a more educated purchase. 
Please direct me to some similar EVs from different companies as the model 3 that I can look at before making the choice.
"""

res: NoCompetitorResponse = client.chat.completions.create(
    model="claude-3-5-sonnet-20240620",
    response_model=NoCompetitorResponse,
    max_tokens=1000,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful sales chatbot for Tesla. You will answer questions about our product and help the user make a purchase.",
        },
    {
        "role": "user",
        "content": message,
    },
    ],
    max_retries=2,
)

print(res.response)

I apologize, but as a Tesla sales representative, I'm not able to provide direct comparisons or recommendations for electric vehicles from other companies. However, I'd be happy to provide you with detailed information about the Tesla Model 3 and its unique features that make it stand out in the electric vehicle market. 

The Tesla Model 3 offers exceptional performance, advanced technology, and long-range capabilities. Some key features include:

1. Long range: Up to 358 miles on a single charge (Long Range version)
2. Quick acceleration: 0-60 mph in as little as 3.1 seconds (Performance version)
3. Advanced Autopilot capabilities
4. Minimalist and spacious interior with a 15-inch touchscreen
5. Over-the-air software updates
6. Access to Tesla's extensive Supercharger network

Would you like me to elaborate on any specific aspects of the Model 3 that you're particularly interested in? I can provide more details on performance, technology, safety features, or any other area you'd like 

### Real world example

Analyzing customer service calls with language models

In [41]:
pii_prompt = """
Given a customer service call transcript, validate if the transcript has correctly been redacted of any Personally Identifiable Information (PII).
If there is any PII found, or the transcript is otherwise invalid or truncated, reject the response.
"""

class ValidatedPIIRedaction(BaseModel):
    response: str
    
    @field_validator("response")
    def validate_redacted_pii(cls, v):
        return validate_response(v, pii_prompt)
    
with open("data/transcript_redacted_3.txt", "r") as file:
    transcript = file.read()
    
try:
    res = ValidatedPIIRedaction(response=transcript)
    print("PII validation passed")
except Exception as e:
    print(e)

1 validation error for ValidatedPIIRedaction
response
  Value error, Invalid response: The provided transcript is not a valid customer service call transcript. It appears to be a statement about the absence of PII rather than an actual transcript of a conversation. A proper transcript should contain a dialogue between a customer service representative and a customer, including their exchanges and any relevant context. Without an actual transcript to review, it's impossible to verify if PII has been correctly redacted. [type=value_error, input_value="Transcript:\n\nThere's n...n the given transcript.", input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error
